### Done in colab using T4 GPU architecture  

# Download dependencies

In [ ]:
!pip install grad-cam lime captum

# Imports

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, WeightedRandomSampler
import torchvision
from torchvision import datasets, transforms, models
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from collections import Counter
import numpy as np
import os
from tqdm import tqdm
from torchvision import transforms as T
from PIL import Image
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
import cv2
from lime import lime_image
from skimage.segmentation import slic, mark_boundaries
import gc
import matplotlib.pyplot as plt
from captum.attr import GradientShap
from sklearn.metrics import classification_report
import time
from sklearn.metrics import accuracy_score, f1_score, jaccard_score

# Tune model for galaxy types

### Zip cloud data to local storage

In [ ]:
!cd "/content/drive/MyDrive/Paper" && zip -r model_dataset.zip model/

!cp "/content/drive/MyDrive/Paper/model_dataset.zip" /content/

!unzip /content/model_dataset.zip -d /content/data_model/

data_dir = "/content/data_model/model"

### Loading the ResNet18 from Pytorch and training on local data

In [ ]:

#Optimized ResNet18 Training Script for Galaxy Classification
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Paths (will need to be updated based on your setup)
data_dir = "/content/data_model/model/"
local_model_path = "/content/model.pt"
drive_model_path = "/content/drive/MyDrive/Paper/model/model.pt"

num_classes = 4
batch_size = 128 
epochs = 25
lr = 1e-4

torch.backends.cudnn.benchmark = True 

#Transforms
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    torchvision.transforms.TrivialAugmentWide(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                         (0.2023, 0.1994, 0.2010))
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                         (0.2023, 0.1994, 0.2010))
])

#Loading dataset
valid_class_names = sorted([
    d.name for d in os.scandir(data_dir)
    if d.is_dir() and d.name != '.ipynb_checkpoints'
])

if not valid_class_names:
    raise FileNotFoundError(f"No valid class directories found in {data_dir}")

class_to_idx = {class_name: i for i, class_name in enumerate(valid_class_names)}

#Collect samples
all_samples = []
valid_exts = ('.jpg')
for class_name in valid_class_names:
    class_path = os.path.join(data_dir, class_name)
    target = class_to_idx[class_name]
    for img_file in os.listdir(class_path):
        if img_file.lower().endswith(valid_exts):
            all_samples.append((os.path.join(class_path, img_file), target))

#Custom Dataset
class CustomImageDataset(torch.utils.data.Dataset):
    def __init__(self, samples, transform=None):
        self.samples = samples
        self.transform = transform

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label

#Splits
train_size = int(0.8 * len(all_samples))
val_size = len(all_samples) - train_size
train_samples, val_samples = torch.utils.data.random_split(all_samples, [train_size, val_size])

train_ds = CustomImageDataset(train_samples, transform=train_transform)
val_ds = CustomImageDataset(val_samples, transform=val_transform)

#Sampling
all_targets = [target for _, target in all_samples]
class_counts = Counter(all_targets)
class_weights = {cls: 1.0 / count for cls, count in class_counts.items()}

train_targets = [target for _, target in train_ds.samples]
sample_weights = [class_weights[t] for t in train_targets]

sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)

train_loader = DataLoader(train_ds, batch_size=batch_size, sampler=sampler,
                          num_workers=4, pin_memory=True, persistent_workers=True)
val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False,
                        num_workers=4, pin_memory=True, persistent_workers=True)

#Model definition
model = models.resnet18(weights="IMAGENET1K_V1")
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

#Loss, optimizer and scheduler
counts = np.array([class_counts.get(i, 0) for i in range(num_classes)])
weights = torch.tensor(sum(counts) / (counts + 1e-8), dtype=torch.float32).to(device)

criterion = nn.CrossEntropyLoss(weight=weights)
optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5)

scaler = torch.amp.GradScaler()

#Training
best_acc = 0
print(f"Starting training for {epochs} epochs -> ")

for epoch in range(epochs):
    model.train()
    train_loss = 0.0

    for imgs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} "):
        imgs, labels = imgs.to(device, non_blocking=True), labels.to(device, non_blocking=True)

        optimizer.zero_grad(set_to_none=True)
        with torch.amp.autocast(device_type='cuda', dtype=torch.float16):
            outputs = model(imgs)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        train_loss += loss.item()

    #Validation
    model.eval()
    correct, total, val_loss = 0, 0, 0
    all_preds, all_labels = [], []

    with torch.no_grad(), torch.amp.autocast(device_type='cuda', dtype=torch.float16):
        for imgs, labels in tqdm(val_loader, desc=f"Epoch {epoch+1}/{epochs} (Val)"):
            imgs, labels = imgs.to(device, non_blocking=True), labels.to(device, non_blocking=True)
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    val_acc = correct / total
    scheduler.step(val_loss)

    try:
        val_f1 = f1_score(all_labels, all_preds, average='weighted')
        val_jaccard = jaccard_score(all_labels, all_preds, average='weighted')
    except ValueError:
        val_f1 = float('nan')
        val_jaccard = float('nan')

    print(f"Epoch {epoch+1}: "
          f"train_loss={train_loss/len(train_loader):.4f}, "
          f"val_loss={val_loss/len(val_loader):.4f}, "
          f"val_acc={val_acc:.4f}, "
          f"val_f1={val_f1:.4f}, "
          f"val_jaccard={val_jaccard:.4f}")

    #Save best model
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), local_model_path)
        print(f"New best model saved (acc={best_acc:.4f})")

#Loading onto drive
!rsync -a "{local_model_path}" "{drive_model_path}"
print(f"Model transferred to Drive: {drive_model_path}")
print(f"Training complete. Best validation accuracy: {best_acc:.4f}")

### Reloading model and checking paths

In [ ]:
#Load model
model = models.resnet18(weights=None) 
num_classes = 4 
model.fc = nn.Linear(model.fc.in_features, num_classes)
model.load_state_dict(torch.load("/content/drive/MyDrive/Paper/model/model.pt"))
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"Using device: {device}")

#Transform 
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                         (0.2023, 0.1994, 0.2010))
])

#Dataset and loader
data_dir = "/content/drive/MyDrive/Paper/galaxy" 
all_samples = []
valid_class_names = sorted([d.name for d in os.scandir(data_dir) if d.is_dir() and d.name != '.ipynb_checkpoints'])
class_to_idx = {class_name: i for i, class_name in enumerate(valid_class_names)}

for class_folder in valid_class_names:
    class_path = os.path.join(data_dir, class_folder)
    target = class_to_idx[class_folder]

    for img_file in os.listdir(class_path):
        img_path = os.path.join(class_path, img_file)
        if os.path.isfile(img_path) and os.path.splitext(img_path)[1].lower() in ['.jpg', '.jpeg', '.png', '.ppm', '.bmp', '.pgm', '.tif', '.tiff', '.webp']:
             all_samples.append((img_path, target))

if not all_samples:
    raise FileNotFoundError(f"No valid image files found in {data_dir}")

#Creating custom dataset
class CustomImageDataset(torch.utils.data.Dataset):
    def __init__(self, samples, transform=None, classes=None, class_to_idx=None):
        self.samples = samples
        self.transform = transform
        self.classes = classes if classes is not None else sorted(list(set([os.path.basename(os.path.dirname(path)) for path, _ in samples])))
        self.class_to_idx = class_to_idx if class_to_idx is not None else {cls_name: i for i, cls_name in enumerate(self.classes)}

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label

dataset = CustomImageDataset(all_samples, transform=transform, classes=valid_class_names, class_to_idx=class_to_idx)
loader = DataLoader(dataset, batch_size=1, shuffle=False)

print("Classes:", dataset.classes)
print(f"Total images: {len(dataset)}")

### Grad_Cam

In [ ]:
#Load model
model = models.resnet18(weights=None)
num_classes = 4
model.fc = nn.Linear(model.fc.in_features, num_classes)
model.load_state_dict(torch.load("/content/drive/MyDrive/Paper/model/model.pt"))
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"Using device: {device}")

#Folders
data_dir = "/content/drive/MyDrive/Paper/galaxy" 
save_folder = "/content/drive/MyDrive/Paper/data/gradcam"
os.makedirs(save_folder, exist_ok=True)

#Transform images to feed into model
transform = T.Compose([
    T.Resize((224,224)),
    T.ToTensor(),
    T.Normalize((0.4914,0.4822,0.4465),
                (0.2023,0.1994,0.2010))
])

#Grad-CAM setup
target_layer = model.layer4[-1]
gradcam = GradCAM(model=model, target_layers=[target_layer])


#Loop through folders in data directory
for class_folder in sorted(os.listdir(data_dir)):
    class_path = os.path.join(data_dir, class_folder)
    if not os.path.isdir(class_path) or class_folder == '.ipynb_checkpoints':
        continue

    #Create save subdirectory
    save_class_folder = os.path.join(save_folder, class_folder)
    os.makedirs(save_class_folder, exist_ok=True)


    for img_file in sorted(os.listdir(class_path)):
        img_path = os.path.join(class_path, img_file)
        pil_img = Image.open(img_path).convert("RGB")
        input_tensor = transform(pil_img).unsqueeze(0).to(device)

        #Target predicted class
        with torch.no_grad():
            outputs = model(input_tensor)
            predicted_class = torch.argmax(outputs, dim=1)
            target_category = predicted_class.item()

        #Define target for Grad-CAM
        targets = [ClassifierOutputTarget(target_category)]

        #Visualization
        img_np = np.array(pil_img).astype(np.float32) / 255.0

        #Run Grad-CAM
        cam = gradcam(input_tensor=input_tensor, targets=targets)
        grayscale_cam = cam[0, :]

        #Resize to match dimensions
        grayscale_cam_resized = cv2.resize(grayscale_cam, (img_np.shape[1], img_np.shape[0]))

        #Overlay
        cam_image = show_cam_on_image(img_np, grayscale_cam_resized, use_rgb=True)

        #Save heatmap
        save_path = os.path.join(save_class_folder, img_file)
        cam_pil = Image.fromarray(cam_image)
        cam_pil.save(save_path)

        print(f"Saved {save_path}")

print("Grad-CAM processing complete for 4-classes. ")

### LIME

In [ ]:
model = models.resnet18(weights=None)
num_classes = 4
model.fc = nn.Linear(model.fc.in_features, num_classes)
model.load_state_dict(torch.load("/content/drive/MyDrive/Paper/model/model.pt"))
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"Using device: {device}")

#Folders
data_dir = "/content/drive/MyDrive/Paper/galaxy" 
save_folder = "/content/drive/MyDrive/Paper/data/lime"
os.makedirs(save_folder, exist_ok=True)

#Transform
transform = T.Compose([
    T.Resize((224,224)),
    T.ToTensor(),
    T.Normalize((0.4914,0.4822,0.4465),
                (0.2023,0.1994,0.2010))
])

#LIME function
def batch_predict(images):
    model.eval()
    processed_images = []
    for img in images:
        if img.shape[2] == 4:
             img = img[:, :, :3]
        #Convert numpy array to PIL Image
        img_pil = Image.fromarray(img.astype(np.uint8))
        processed_images.append(transform(img_pil))

    batch = torch.stack(processed_images, dim=0).to(device)

    with torch.no_grad():
        logits = model(batch)
        probs = torch.nn.functional.softmax(logits, dim=1)
    return probs.cpu().numpy()

explainer = lime_image.LimeImageExplainer()

#Process a single image
def explain_image(img_path, class_folder, num_classes):
    try:
        pil_img = Image.open(img_path).convert("RGB")
        img_np = np.array(pil_img)

        #Get predicted class
        img_tensor = transform(pil_img).unsqueeze(0).to(device)
        with torch.no_grad():
            outputs = model(img_tensor)
            predicted_class = torch.argmax(outputs, dim=1).item()


        explanation = explainer.explain_instance(
            img_np,
            batch_predict,
            top_labels=num_classes,
            labels=[predicted_class],
            hide_color=0,
            num_samples=200,
            segmentation_fn=lambda x: slic(x, n_segments=50, compactness=10, sigma=1)
        )

        #Get image and mask
        temp, mask = explanation.get_image_and_mask(
            label=predicted_class,
            positive_only=True,
            hide_rest=False,
            num_features=10
        )

        #Visualize
        plt.figure(figsize=(6,6))
        plt.imshow(mark_boundaries(temp / 255.0, mask))
        plt.title(f"LIME Explanation for Predicted Class: {predicted_class}")
        plt.axis('off')

        #Save to subdirectory
        save_class_folder = os.path.join(save_folder, class_folder)
        os.makedirs(save_class_folder, exist_ok=True)
        save_path = os.path.join(save_class_folder, os.path.basename(img_path))
        plt.savefig(save_path, bbox_inches='tight', pad_inches=0)
        plt.close()

        #Free memory
        del pil_img, img_np, explanation, temp, mask, img_tensor, outputs
        gc.collect()
        torch.cuda.empty_cache()

        return f"Saved {save_path}"
    except Exception as e:
        return f"Failed {img_path}: {e}"

#Process images sequentially
for class_folder in sorted(os.listdir(data_dir)):
    class_path = os.path.join(data_dir, class_folder)
    if not os.path.isdir(class_path) or class_folder == '.ipynb_checkpoints':
        continue

    for img_file in sorted(os.listdir(class_path)):
        img_path = os.path.join(class_path, img_file)
        print(explain_image(img_path, class_folder, num_classes))

print("LIME processing complete for 4-classes. ")

### DeepShap using GradeintShap

In [ ]:
model = models.resnet18(weights=None)
num_classes = 4 
model.fc = nn.Linear(model.fc.in_features, num_classes)
model.load_state_dict(torch.load("/content/drive/MyDrive/Paper/model/model.pt"))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"Using device: {device}")


#Replace in-place ReLU operations
def replace_inplace_relu(model):
    for name, module in model.named_children():
        if isinstance(module, torch.nn.ReLU):
            setattr(model, name, torch.nn.ReLU(inplace=False))
        else:
            replace_inplace_relu(module)

replace_inplace_relu(model)
print("Replaced in-place ReLU operations.")

#Transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                         (0.2023, 0.1994, 0.2010))
])

#Paths
image_folder = "/content/drive/MyDrive/Paper/galaxy"
save_folder = "/content/drive/MyDrive/Paper/data/deepshap"
os.makedirs(save_folder, exist_ok=True)


#GradientShap explainer
num_baselines = 10
baselines = torch.randn(num_baselines, 3, 224, 224).to(device) * 0.05

explainer = GradientShap(model)

# Loop through original data
for class_folder in sorted(os.listdir(image_folder)):
    class_path = os.path.join(image_folder, class_folder)
    if not os.path.isdir(class_path) or class_folder == '.ipynb_checkpoints':
        continue

    #Save subdirectory
    save_class_folder = os.path.join(save_folder, class_folder)
    os.makedirs(save_class_folder, exist_ok=True)

    for img_file in sorted(os.listdir(class_path)):
        img_path = os.path.join(class_path, img_file)
        pil_img = Image.open(img_path).convert("RGB")

        #Preprocess image
        input_tensor = transform(pil_img).unsqueeze(0).to(device)
        input_tensor.requires_grad = True

        with torch.no_grad():
            outputs = model(input_tensor)
            predicted_class = torch.argmax(outputs, dim=1)
            target_category = predicted_class.item()

        #Compute attributions
        attributions = explainer.attribute(
            input_tensor,
            baselines=baselines,
            target=target_category, 
            stdevs=0.0001
        )

        #Convert to numpy
        attr = attributions.squeeze().detach().cpu().numpy()
        attr = np.sum(attr, axis=0)
        attr = np.abs(attr)

        #Visualization and overlay
        attr = (attr - attr.min()) / (attr.max() - attr.min() + 1e-8)
        img_np_resized = np.array(pil_img.resize((224, 224))) / 255.0
        heatmap = plt.cm.inferno(attr)[:, :, :3]
        overlay = 0.5 * img_np_resized + 0.5 * heatmap

        #Save image
        save_path = os.path.join(save_class_folder, img_file)
        plt.imsave(save_path, overlay)
        print(f"Saved {save_path}")

        #Free memory
        del input_tensor, attributions, attr, img_np_resized, heatmap, overlay
        torch.cuda.empty_cache()


print("GradientShap attribution complete for 4-classes.")

# Metrics



### Runtime, fidelity curves and FES check


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load model
model = models.resnet18(weights=None)
num_classes = 4
model.fc = nn.Linear(model.fc.in_features, num_classes)
model.load_state_dict(torch.load("/content/drive/MyDrive/Paper/model/model.pt"))
model.eval()
model.to(device)

#Folders
save_folders = {
    "deepshap": "/content/drive/MyDrive/Paper/data/deepshap",
    "lime": "/content/drive/MyDrive/Paper/data/lime",
    "gradcam": "/content/drive/MyDrive/Paper/data/gradcam"
}

#Transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                         (0.2023, 0.1994, 0.2010))
])


#Evaluate fidelity and runtime
def evaluate_fidelity_curve(model, data_dir, attr_root_folder, thresholds=[0.05, 0.1, 0.15, 0.25, 0.5]):
    results = {t: [] for t in thresholds}
    runtimes = []

    #Create samples
    all_samples = []
    # List valid class names by checking directories in data_dir and excluding .ipynb_checkpoints
    valid_class_names = sorted([d.name for d in os.scandir(data_dir) if d.is_dir() and d.name != '.ipynb_checkpoints'])
    class_to_idx = {class_name: i for i, class_name in enumerate(valid_class_names)}

    for class_folder in valid_class_names:
        class_path = os.path.join(data_dir, class_folder)
        target = class_to_idx[class_folder]

        for img_file in os.listdir(class_path):
            img_path = os.path.join(class_path, img_file)
            if os.path.isfile(img_path) and os.path.splitext(img_path)[1].lower() in ['.jpg']:
                 all_samples.append((img_path, target))

    if not all_samples:
        raise FileNotFoundError(f"No valid image files found in {data_dir}")

    class CustomImageDataset(torch.utils.data.Dataset):
        def __init__(self, samples, transform=None, classes=None, class_to_idx=None):
            self.samples = samples
            self.transform = transform
            self.classes = classes if classes is not None else sorted(list(set([os.path.basename(os.path.dirname(path)) for path, _ in samples])))
            self.class_to_idx = class_to_idx if class_to_idx is not None else {cls_name: i for i, cls_name in enumerate(self.classes)}

        def __len__(self):
            return len(self.samples)

        def __getitem__(self, idx):
            img_path, label = self.samples[idx]
            image = Image.open(img_path).convert("RGB")
            if self.transform:
                image = self.transform(image)
            return image, label


    #Process each sample
    for img_path, true_label in tqdm(all_samples, desc=f"Evaluating {os.path.basename(attr_root_folder)}"):
        start_time = time.time()

        try:
            pil_img = Image.open(img_path).convert("RGB")
            img_np = np.array(pil_img) / 255.0
            img_np_resized = cv2.resize(img_np, (224, 224))
            img_tensor = transform(Image.fromarray((img_np_resized * 255).astype(np.uint8))).unsqueeze(0).to(device)
        except Exception as e:
            print(f"⚠️ Error loading image {img_path}: {e}")
            continue

        #Prediction on full image
        with torch.no_grad():
            full_out = model(img_tensor)
            #True class probability
            full_prob = torch.softmax(full_out, dim=1)[0, true_label].item()


        #Corresponding attribution heatmap
        original_class_folder = os.path.basename(os.path.dirname(img_path))
        attr_class_folder_path = os.path.join(attr_root_folder, original_class_folder)
        attr_file_name = os.path.basename(img_path).split('.')[0] + '.jpg'
        attr_path = os.path.join(attr_class_folder_path, attr_file_name)


        if not os.path.exists(attr_path):
            print(f"⚠️ Missing heatmap for {img_path} in {attr_class_folder_path}")
            continue


        try:
            attr = cv2.imread(attr_path, cv2.IMREAD_GRAYSCALE) / 255.0
            attr = cv2.resize(attr, (224, 224))
        except Exception as e:
             print(f"⚠️ Error loading heatmap {attr_path}: {e}")
             continue


        flat = attr.flatten()
        for t in thresholds:
            k = int(len(flat) * t)
            thresh_val = np.sort(flat)[-k] if k > 0 else (np.max(flat) + 1e-8)
            mask = (attr >= thresh_val).astype(np.float32)

            masked_img_resized = img_np_resized * mask[:, :, None]
            masked_tensor = transform(Image.fromarray((masked_img_resized * 255).astype(np.uint8))).unsqueeze(0).to(device)

            with torch.no_grad():
                masked_out = model(masked_tensor)
                masked_prob = torch.softmax(masked_out, dim=1)[0, true_label].item()

            drop = full_prob - masked_prob
            results[t].append(drop)

        runtimes.append(time.time() - start_time)


    mean_fidelity = {t: np.mean(v) for t, v in results.items() if len(v) > 0}
    mean_runtime = np.mean(runtimes) if runtimes else 0

    return mean_fidelity, mean_runtime


#Run for all explainers
fidelity_curves = {}
runtime_results = {}
fes_scores = {}
data_dir = "/content/drive/MyDrive/Paper/galaxy"

for explainer, folder in save_folders.items():
    print(f"⏳ Evaluating {explainer}...")
    fidelity_curves[explainer], runtime_results[explainer] = evaluate_fidelity_curve(model, data_dir, folder)
    print(f"✅ {explainer}: Avg runtime {runtime_results[explainer]:.3f}s per image")

    #FES score
    mean_fid = np.mean(list(fidelity_curves[explainer].values())) if fidelity_curves[explainer] else 0
    mean_runtime = runtime_results[explainer]
    fes_scores[explainer] = mean_fid / mean_runtime if mean_runtime > 0 else 0


#Plot Fidelity Curves
plt.figure(figsize=(7,5))
for explainer, curve in fidelity_curves.items():
    plt.plot(list(curve.keys()), list(curve.values()), marker='o', label=explainer)
plt.xlabel("Top % Pixels Retained")
plt.ylabel("Mean Confidence Drop (True Class)")
plt.title("Fidelity Curves for Explainability Methods (4-Class Classification)")
plt.legend()
plt.grid(True)
plt.show()

#Plot Runtime Comparison
plt.figure(figsize=(6,4))
plt.bar(runtime_results.keys(), runtime_results.values(), color=['red','green','blue'])
plt.ylabel("Seconds per Image")
plt.title("Runtime Comparison (Lower is Better)")
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

#Plot FES Comparison
plt.figure(figsize=(6,4))
plt.bar(fes_scores.keys(), fes_scores.values(), color=['purple','orange','teal'])
plt.ylabel("Fidelity–Efficiency Score (Higher = Better)")
plt.title("Overall Explainability Performance (4-Class Classification)")
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

#Print Summary
print("🏁 Fidelity + Runtime + FES analysis complete.")
print("\n🔍 Fidelity–Efficiency Scores:")
for k, v in fes_scores.items():
    print(f"{k:<20}: {v:.4f}")

### Weighted F1 Score and Jaccard Score

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

#Load model 
model = models.resnet18(weights=None)
num_classes = 4
model.fc = nn.Linear(model.fc.in_features, num_classes)
model.load_state_dict(torch.load("/content/drive/MyDrive/Paper/model/model.pt"))
model.eval()
model.to(device)


#Load dataset
data_dir = "/content/drive/MyDrive/Paper/galaxy" 

all_samples = []
#Valid class names
valid_class_names = sorted([d.name for d in os.scandir(data_dir) if d.is_dir() and d.name != '.ipynb_checkpoints'])
class_to_idx = {class_name: i for i, class_name in enumerate(valid_class_names)}

for class_folder in valid_class_names:
    class_path = os.path.join(data_dir, class_folder)
    target = class_to_idx[class_folder]

    for img_file in os.listdir(class_path):
        img_path = os.path.join(class_path, img_file)
        if os.path.isfile(img_path) and os.path.splitext(img_path)[1].lower() in ['.jpg']:
             all_samples.append((img_path, target))

if not all_samples:
    raise FileNotFoundError(f"No valid image files found in {data_dir}")

class CustomImageDataset(torch.utils.data.Dataset):
    def __init__(self, samples, transform=None, classes=None, class_to_idx=None):
        self.samples = samples
        self.transform = transform
        self.classes = classes if classes is not None else sorted(list(set([os.path.basename(os.path.dirname(path)) for path, _ in samples])))
        self.class_to_idx = class_to_idx if class_to_idx is not None else {cls_name: i for i, cls_name in enumerate(self.classes)}

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label


dataset = CustomImageDataset(all_samples, transform=transform, classes=valid_class_names, class_to_idx=class_to_idx)
dataloader = DataLoader(dataset, batch_size=32, shuffle=False)

#Labels
true_labels = []
predicted_labels = []

#Evaluate model
model.eval()
with torch.no_grad():
    for images, labels in dataloader:
        images = images.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)

        true_labels.extend(labels.cpu().numpy())
        predicted_labels.extend(preds.cpu().numpy())

#Calculate metrics
overall_accuracy = accuracy_score(true_labels, predicted_labels)
f1_weighted = f1_score(true_labels, predicted_labels, average='weighted')
jaccard_weighted = jaccard_score(true_labels, predicted_labels, average='weighted')

print(f"Overall Accuracy: {overall_accuracy:.4f}")
print(f"F1 Score (weighted): {f1_weighted:.4f}")
print(f"Jaccard Similarity (weighted): {jaccard_weighted:.4f}")

print("\nClass-wise Metrics:")
print(classification_report(true_labels, predicted_labels, target_names=dataset.classes))

#Free memory
del true_labels, predicted_labels, dataset, dataloader, model, images, labels, outputs, preds
gc.collect()
torch.cuda.empty_cache()